### Word Embeddings
Embedding layer can be used to perform three task in Keras:
* learn word embeddings and save the resulting model
* learn word embeddings in addition to perform the NLP task such as text classification, sentiment analysis, etc.
* load pretrained word embeddings and use them in new models

Example:

`embedding_layer = Embedding(200,32,input_length=50)`

First param: size of vocabulary or total number of unique words in corpus.
Second param: number of dimensions for each word vector.
Third param: length of input sentence.

Output of the word embedding is a 2D vector where words are represented in rows and their corresponding dimensions are presented in columns.

In [13]:
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

2021-08-24 22:39:54.740611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-24 22:39:54.740667: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]


In [15]:
sentiments = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [17]:
unique_words = set(all_words)
print(len(unique_words))

45


In [18]:
vocab_length = 50
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]

In [19]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [20]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[ 6  6 44 39 47  0  0]
 [ 4 43 29 35 32 49 28]
 [49 14 10 28  6 20  0]
 [42 22  0  0  0  0  0]
 [38 15 24 39 32 49 28]
 [44 33 35 16  0  0  0]
 [32 35 28 33 38 47  0]
 [28  6 33 10 47  0  0]
 [26  7  0  0  0  0  0]
 [26  8 16  0  0  0  0]
 [18 49  0  0  0  0  0]
 [28 29  7  3  0  0  0]
 [32 11 18 49  4 47  0]
 [ 4 47 29 26  0  0  0]
 [32 45 18 47  0  0  0]
 [ 4  7  6 18  0  0  0]]


In [21]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

2021-08-24 22:42:46.380222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-24 22:42:46.380297: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-24 22:42:46.380352: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rx): /proc/driver/nvidia/version does not exist
2021-08-24 22:42:46.381024: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 20)             1000      
_________________________________________________________________
flatten (Flatten)            (None, 140)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


You can see that the first layer has 1000 trainable parameters. This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuron


In [23]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

2021-08-24 22:43:11.747912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
1/1 [==============================] - 1s 717ms/step - loss: 0.6893 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6858 - acc: 0.6250
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6824 - acc: 0.6250
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6789 - acc: 0.6250
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6755 - acc: 0.6875
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6721 - acc: 0.8125
Epoch 7/100
1/1 [=====

In [24]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy * 100))

Accuracy: 100.000000
